# Libraries

**Elasticsearch Query**

In [1]:
import sys
pathModulesES = '../sauceforyall/'
sys.path.append(pathModulesES)

In [2]:
from yelpquery import YelpQuery
from pandasticsearch import Select
ye = YelpQuery()

**Machine Learning**

In [3]:
import pandas as pd
import numpy as np
import nltk
import pickle
from scipy import sparse

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

**Spark**

In [6]:
import findspark
findspark.init("/home/hongphuc95/anaconda3/lib/python3.7/site-packages/pyspark")

In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import DoubleType, IntegerType, StringType

In [10]:
spark = SparkSession.builder \
                    .appName('yelp_server') \
                    .master("local[*]") \
                    .getOrCreate()

**Visualize**

In [11]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

**Others**

In [12]:
import re
import string

**Index name**

In [8]:
index_business = "yelp-business*"
index_review = "yelp-review*"
index_tip = "yelp-tip*"
index_user = "yelp-user*"

In [5]:
data_path = "/home/hongphuc95/notebookteam/dataset/"

# 1. Data Pre-Processing

## 1.1 Load and clean data

In [8]:
business_df = pd.read_json(data_path + "business.json", lines=True)

In [ ]:
#review_df = pd.read_json(data_path + "cleaned/review_cleaned_2016_2019_Vegas.json", lines=True)

In [14]:
review_df.shape

(1041555, 9)

**Missing data**

In [15]:
missing_cat_business = business_df[business_df.isnull()]["business_id"].tolist()
business_df = business_df.dropna(subset=["categories"])

**To avoid coldstart problem, we filter out all users have less than 10 reviews**

In [16]:
user_review_df_count = review_df.groupby(["user_id", "business_id"]).size().groupby("user_id").size()
user_review_active_df = user_review_df_count[user_review_df_count > 20].reset_index()[["user_id"]]

In [17]:
user_active_df = review_df[review_df["user_id"].isin(user_review_active_df["user_id"])]

In [18]:
user_active_df.shape

(173597, 9)

**Clean attributes column**

In [19]:
import ast

def fusion_attr(key, val):
    attr_full = ""
    key = key.replace("'","")
    sub_string = re.search('u\'(\w*)\'', val)
    if sub_string:
        attr_full = key + "_" + sub_string.group(1)
    else:
        val = val.replace("'","")
        attr_full = key + "_" + val
    return attr_full
    
def flatten_attr(nested_field):
    l=[]
    if nested_field:
        for (idx,val) in nested_field.items():
            f_level = idx
            s_level = re.findall(r"{(.+)}", val)    
            if s_level:
                for element in s_level[0].split(", "):
                    element_splited = element.split(":")
                    key_element = f_level + "_" + element_splited[0].strip()
                    val_element = element_splited[1].strip()
                    to_add_string = fusion_attr(key_element, val_element)
                    l.append(to_add_string)
            else:
                to_add_string = fusion_attr(f_level, val)
                l.append(to_add_string)    
    return ", ".join(l)

In [20]:
business_df['attributes'] = business_df['attributes'].apply(lambda x: flatten_attr(x))

**Mean rating**

In [19]:
user_active_df = user_active_df.groupby(["user_id", "business_id"], as_index=False).mean()

In [20]:
user_active_df.shape

(165496, 6)

**Encoding business_id and user_id for the sparse matrix**

In [21]:
from sklearn.preprocessing import LabelEncoder

In [22]:
user_encoder = LabelEncoder()
business_encoder = LabelEncoder()

In [23]:
user_ids = pd.DataFrame(columns=["user_id_matrix"])
business_ids = pd.DataFrame(columns=["business_id_matrix"])

In [24]:
#user_ids["user_id"] = user_active_df["user_id"].unique()
#business_ids["business_id"] = user_active_df["business_id"].unique()

In [25]:
user_active_df["user_id_matrix"] = user_encoder.fit_transform(user_active_df['user_id'])
user_active_df["business_id_matrix"] = business_encoder.fit_transform(user_active_df['business_id'])

In [26]:
user_ids["user_id_matrix"] = user_active_df["user_id_matrix"].unique()
business_ids["business_id_matrix"] = user_active_df["business_id_matrix"].unique()
user_ids["user_id"] = user_encoder.inverse_transform(user_ids["user_id_matrix"])
business_ids["business_id"] = business_encoder.inverse_transform(business_ids["business_id_matrix"])

In [27]:
#user_active_df.to_json("/home/hongphuc95/notebookteam/dataset/cleaned/user_active.json", orient='records', lines=True)

# 1.2 Before going in

In [30]:
business_df.head(5)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,1SWheh84yJXfytovILXOAQ,Arizona Biltmore Golf Club,2818 E Camino Acequia Drive,Phoenix,AZ,85016,33.522143,-112.018481,3.0,5,0,GoodForKids_False,"Golf, Active Life",None
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,128,1,"RestaurantsReservations_True, GoodForMeal_dess...","Specialty Food, Restaurants, Dim Sum, Imported...","{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W..."
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132,4.0,170,1,"GoodForKids_True, NoiseLevel_average, Restaura...","Sushi Bars, Restaurants, Japanese","{'Monday': '17:30-21:30', 'Wednesday': '17:30-..."
3,xvX2CttrVhyG2z1dFg_0xw,Farmers Insurance - Paul Lorenz,"15655 W Roosevelt St, Ste 237",Goodyear,AZ,85338,33.455613,-112.395596,5.0,3,1,,"Insurance, Financial Services","{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ..."
4,HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,"4209 Stuart Andrew Blvd, Ste F",Charlotte,NC,28217,35.190012,-80.887223,4.0,4,1,"BusinessAcceptsBitcoin_False, ByAppointmentOnl...","Plumbing, Shopping, Local Services, Home Servi...","{'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', ..."


In [31]:
business_df.shape

(192127, 14)

In [32]:
user_active_df.head(5)

,user_id,business_id,cool,useful,funny,stars,user_id_matrix,business_id_matrix
0,--2vR0DIsmQ6WfcSzKWigw,3xmfT7l3xNH5LK1dLzfvGw,207.0,207.0,181.0,4.0,0,1460
1,--2vR0DIsmQ6WfcSzKWigw,4ONpzAtnKbDig_e_O_8TtQ,202.0,203.0,179.0,5.0,0,1586
2,--2vR0DIsmQ6WfcSzKWigw,7dHYudt6OOIjiaxkSvv3lQ,200.0,207.0,183.0,5.0,0,2609
3,--2vR0DIsmQ6WfcSzKWigw,7fsAew2vBXJ05QlOu9d_lA,199.0,209.0,177.0,3.0,0,2618
4,--2vR0DIsmQ6WfcSzKWigw,9edPSkfXKsJmkZYIaOmA7Q,198.0,204.0,181.0,4.0,0,3273


# Building Evaluator 

In [33]:
user_active_train_df, user_active_test_df = train_test_split(user_active_df,
                                   stratify=user_active_df['user_id'], 
                                   test_size=0.20,
                                   random_state=42)


In [34]:
print('Train set: %d' % len(user_active_train_df))
print('Test set: %d' % len(user_active_test_df))

Train set: 221312
Test set: 55329


In [35]:
#Indexing by personId to speed up the searches during evaluation
user_active_indexed_df = user_active_df.set_index('user_id')
user_active_train_indexed_df = user_active_train_df.set_index('user_id')
user_active_test_indexed_df = user_active_test_df.set_index('user_id')

In [36]:
def get_items_reviews(user_id, user_active_df):
    # Get the user's data and merge in the business information.
    reviewed_items = user_active_df.loc[user_id]['business_id']
    return set(reviewed_items if type(reviewed_items) == pd.Series else [reviewed_items])

In [37]:
#Top-N accuracy metrics consts
EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS = 100

class ModelEvaluator:


    def get_not_reviewed_items_sample(self, user_id, sample_size, seed=42):
        reviewed_items = get_items_reviews(user_id, user_active_indexed_df)
        all_items = set(business_df['business_id'])
        non_reviewed_items = all_items - reviewed_items

        random.seed(seed)
        non_reviewed_items_sample = random.sample(non_reviewed_items, sample_size)
        return set(non_reviewed_items_sample)

    def _verify_hit_top_n(self, item_id, recommended_items, topn):        
            try:
                index = next(i for i, c in enumerate(recommended_items) if c == item_id)
            except:
                index = -1
            hit = int(index in range(0, topn))
            return hit, index

    def evaluate_model_for_user(self, model, user_id):
        #Getting the items in test set
        reviewed_values_testset = user_active_test_indexed_df.loc[user_id]
        if type(reviewed_values_testset['business_id']) == pd.Series:
            user_reviewed_items_testset = set(reviewed_values_testset['business_id'])
        else:
            user_reviewed_items_testset = set([int(reviewed_values_testset['business_id'])])  
        reviewed_items_count_testset = len(user_reviewed_items_testset) 

        #Getting a ranked recommendation list from a model for a given user
        user_recs_df = model.recommend(user_id, 
                                               items_to_ignore=get_items_reviews(user_id, 
                                                                                    user_active_test_indexed_df), 
                                               topn=10000000000)

        hits_at_5_count = 0
        hits_at_10_count = 0
        #For each item the user has interacted in test set
        for item_id in user_reviewed_items_testset:
            #Getting a random sample (100) items the user has not interacted 
            #(to represent items that are assumed to be no relevant to the user)
            non_reviewed_items_sample = self.get_not_reviewed_items_sample(user_id, 
                                                                          sample_size=EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS, 
                                                                          seed=42)

            #Combining the current interacted item with the 100 random items
            items_to_filter_recs = non_reviewed_items_sample.union(set([item_id]))

            #Filtering only recommendations that are either the interacted item or from a random sample of 100 non-interacted items
            valid_recs_df = user_recs_df[user_recs_df['business_id'].isin(items_to_filter_recs)]                    
            valid_recs = valid_recs_df['business_id'].values
            #Verifying if the current interacted item is among the Top-N recommended items
            hit_at_5, index_at_5 = self._verify_hit_top_n(item_id, valid_recs, 5)
            hits_at_5_count += hit_at_5
            hit_at_10, index_at_10 = self._verify_hit_top_n(item_id, valid_recs, 10)
            hits_at_10_count += hit_at_10

        #Recall is the rate of the interacted items that are ranked among the Top-N recommended items, 
        #when mixed with a set of non-relevant items
        recall_at_5 = hits_at_5_count / float(reviewed_items_count_testset)
        recall_at_10 = hits_at_10_count / float(reviewed_items_count_testset)

        user_metrics = {'hits@5_count':hits_at_5_count, 
                          'hits@10_count':hits_at_10_count, 
                          'reviewed_count': reviewed_items_count_testset,
                          'recall@5': recall_at_5,
                          'recall@10': recall_at_10}
        print(user_metrics)
        return user_metrics

    def evaluate_model(self, model):
        #print('Running evaluation for users')
        people_metrics = []
        for idx, user_id in enumerate(list(user_active_test_indexed_df.index.unique().values)):
            #if idx % 100 == 0 and idx > 0:
            #    print('%d users processed' % idx)
            user_metrics = self.evaluate_model_for_user(model, user_id)  
            user_metrics['_user_id'] = user_id
            people_metrics.append(user_metrics)
        print('%d users processed' % idx)

        detailed_results_df = pd.DataFrame(people_metrics) \
                            .sort_values('reviewed_count', ascending=False)
        
        global_recall_at_5 = detailed_results_df['hits@5_count'].sum() / float(detailed_results_df['reviewed_count'].sum())
        global_recall_at_10 = detailed_results_df['hits@10_count'].sum() / float(detailed_results_df['reviewed_count'].sum())
        
        global_metrics = {'modelName': model.get_model_name(),
                          'recall@5': global_recall_at_5,
                          'recall@10': global_recall_at_10}    
        return global_metrics, detailed_results_df

In [38]:
model_evaluator = ModelEvaluator() 

# Popularity Based

In [44]:
item_popularity_df = user_active_df.groupby("business_id")["stars"].sum() \
                    .sort_values(ascending=False).reset_index()

In [45]:
item_popularity_df.head(10)

,business_id,stars
0,igHYkXZMLAc9UdV5VnR_AA,2417.500000
1,IhNASEZ3XnBHmuuVnWdIwA,2372.000000
2,RESDUcs7fIiihp38-d6_6g,2203.500000
3,q3oJ6bNRV3OoJrwc95GOwg,2113.666667
4,4JNXUYY8wbaaDmk3BPzlWw,1926.500000
5,eaNenRk_liZBERFFLCXqqQ,1835.500000
6,SVGApDPNdpFlEjwRQThCxA,1824.000000
7,FaHADZARwnY4yvlvpnsfGA,1734.000000
8,gx2yPrOJSwF1ApJYdGBWIw,1683.833333
9,B_WggEKFq-ZFNui8CHPYvA,1515.333333


In [46]:
import random
class PopularityRecommender:
    MODEL_NAME = 'Popularity'
    
    def __init__(self, popularity_df):
        self.popularity_df = popularity_df

    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend(self, items_to_ignore=[], topn=10):
        # Recommend the more popular items that the user hasn't seen yet.
        recommendations_df = self.popularity_df[~self.popularity_df['business_id'].isin(items_to_ignore)] \
                               .sort_values('stars', ascending = False) \
                               .head(topn)
        #recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
        #                                              left_on = 'business_id', 
        #                                              right_on = 'business_id')[["name", "business_id", "stars"]]


        return recommendations_df

In [47]:
popularity_model = PopularityRecommender(item_popularity_df, business_df)

In [48]:
print('Evaluating Popularity recommendation model...')
pop_global_metrics, pop_detailed_results_df = model_evaluator.evaluate_model(popularity_model)

Evaluating Popularity recommendation model...
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 8, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 5, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 54, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 16, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 9, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 3, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 15, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 9, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 5, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 4, 'recall@5': 0.0, 'recall@1

{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 13, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 4, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 3, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 4, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 8, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 6, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 33, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 3, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 3, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 11, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 

{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 4, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 16, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 2, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 2, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 19, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 73, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 13, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 6, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 3, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 17, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count'

{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 38, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 5, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 2, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 7, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 6, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 3, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 18, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 6, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 4, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 21, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 

{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 16, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 4, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 2, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 2, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 12, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 11, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 4, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 3, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 35, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 53, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count'

...

{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 3, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 3, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 2, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 3, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 2, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 2, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 3, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 2, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 3, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 2, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 

{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 3, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 4, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 3, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 3, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 2, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 3, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 2, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 2, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 3, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 2, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 

{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 2, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 3, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 2, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 2, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 3, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 2, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 2, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 2, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 3, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 2, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 

{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 3, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 2, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 2, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 2, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 3, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 2, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 2, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 4, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 3, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 3, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 

{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 2, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 2, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 3, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 2, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 3, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 2, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 2, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 2, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 3, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 'reviewed_count': 2, 'recall@5': 0.0, 'recall@10': 0.0}
{'hits@5_count': 0, 'hits@10_count': 0, 

In [50]:
print('\nGlobal metrics:\n%s' % pop_global_metrics)
pop_detailed_results_df.head(10)


Global metrics:
{'modelName': 'Popularity', 'recall@5': 0.0, 'recall@10': 0.0}


,hits@5_count,hits@10_count,reviewed_count,recall@5,recall@10,_user_id
46,0,0,156,0.0,0.0,bLbSNkLggFnqwNNzzq-Ijw
249,0,0,123,0.0,0.0,I-4KVZ9lqHhk8469X9FvhA
1075,0,0,90,0.0,0.0,JaqcCU3nxReTW2cBLHounA
323,0,0,79,0.0,0.0,keBv05MsMFBd0Hu98vXThQ
176,0,0,73,0.0,0.0,qPVtjjp8sNQ32p9860SR9Q
657,0,0,68,0.0,0.0,PKEzKWv_FktMm2mGPjwd0Q
429,0,0,60,0.0,0.0,YE54kKTuqJJPNYWIKIpOEQ
143,0,0,59,0.0,0.0,8dxkcmAXY4ttrVFD1GhbdQ
464,0,0,58,0.0,0.0,3nIuSCZk5f_2WWYMLN7h3w
2,0,0,54,0.0,0.0,OXSJCjKtvZPf-YPDCXcWZg


# Content Based

In [39]:
vectorizer = TfidfVectorizer(analyzer = 'word', stop_words = 'english', 
                             lowercase = True, max_features = 5000,
                             ngram_range = (1, 1)
                            )

In [40]:
tfidf_matrix = vectorizer.fit_transform(business_df["categories"] + business_df["attributes"])
tfidf_feature_names = vectorizer.get_feature_names()

In [41]:
item_ids = business_df['business_id'].tolist()

In [42]:
from sklearn.preprocessing import normalize
def get_item_profile(item_id):
    idx = item_ids.index(item_id)
    item_profile = tfidf_matrix[idx:idx+1]
    return item_profile

def get_item_profiles(ids):
    item_profiles_list = [get_item_profile(x) for x in ids]
    item_profiles = sparse.vstack(item_profiles_list)
    return item_profiles

def build_users_profile(user_id, user_active_indexed_df):
    user_behaviour_df = user_active_indexed_df.loc[user_id]
    user_item_profiles = get_item_profiles(user_behaviour_df['business_id'])
    user_item_score = np.array(user_behaviour_df['stars']).reshape(-1,1)

    #Weighted average of item profiles by the interactions strength
    user_item_score_weighted_avg = np.sum(user_item_profiles.multiply(user_item_score), axis=0) / np.sum(user_item_score)
    user_profile_norm = normalize(user_item_score_weighted_avg)
    return user_profile_norm

def build_users_profiles(): 
                                        
    user_active_indexed_df = user_active_train_df[user_active_train_df["business_id"].isin(business_df["business_id"])].set_index("user_id")    
    user_profiles = {}
    for user_id in user_active_indexed_df.index.unique():
        user_profiles[user_id] = build_users_profile(user_id, user_active_indexed_df).ravel()
    return user_profiles

In [43]:
user_profiles = build_users_profiles()
len(user_profiles)

11863

In [183]:
user_profiles

{'--2vR0DIsmQ6WfcSzKWigw': array([0., 0., 0., ..., 0., 0., 0.]),
 '--RlSfc-QmcHFGHyX6aVjA': array([0., 0., 0., ..., 0., 0., 0.]),
 '--ZNfWKj1VyVElRx6-g1fg': array([0., 0., 0., ..., 0., 0., 0.]),
 '-05XqtNjcBq19vh2CVJN8g': array([0., 0., 0., ..., 0., 0., 0.]),
 '-0OyT3le0GqdyvRLLfB7MQ': array([0., 0., 0., ..., 0., 0., 0.]),
 '-16aKjco1c0RJ7c4U-q_Kw': array([0., 0., 0., ..., 0., 0., 0.]),
 '-191gKrqDzXGUrpl7npkXw': array([0., 0., 0., ..., 0., 0., 0.]),
 '-1x8gXJnrI-FeZPNvnfbRg': array([0., 0., 0., ..., 0., 0., 0.]),
 '-2gOxVWcnBr5DclrrsWXCA': array([0., 0., 0., ..., 0., 0., 0.]),
 '-3Tgw9br9O68emgrgFj48Q': array([0., 0., 0., ..., 0., 0., 0.]),
 '-3gUn9YImya8KvHKeHMmFQ': array([0., 0., 0., ..., 0., 0., 0.]),
 '-48H9MSOaigzH-v8ytPaDw': array([0., 0., 0., ..., 0., 0., 0.]),
 '-4JBueG4IRIh2Ie5EEIJSg': array([0., 0., 0., ..., 0., 0., 0.]),
 '-4JDJeFS0YAYSiSvIshGLQ': array([0., 0., 0., ..., 0., 0., 0.]),
 '-50XWnmQGqBgEI-9ANvLlg': array([0., 0., 0., ..., 0., 0., 0.]),
 '-57QhzPVMWZXX92YaJ1pdw'

In [207]:
user_profiles_df = pd.DataFrame.from_dict(user_profiles, orient='index')

In [212]:
user_profiles_df.reset_index(level=0, inplace=True)

In [213]:
user_profiles_df.columns = user_profiles_df.columns.astype(str)

In [214]:
user_profiles_df.to_parquet(data_path + "cleaned/user_profiles_df.parquet")

In [217]:
parquetFile = spark.read.parquet(data_path + "cleaned/user_profiles_df.parquet")
parquetFile.createOrReplaceTempView("parquetFile")
test = spark.sql("SELECT * FROM parquetFile WHERE index = '-9BOs3rdWFMot0tlky983Q'").toPandas()

In [222]:
test.set_index('index', inplace=True)

In [224]:
test.loc["-9BOs3rdWFMot0tlky983Q"]

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
4995    0.0
4996    0.0
4997    0.0
4998    0.0
4999    0.0
Name: -9BOs3rdWFMot0tlky983Q, Length: 5000, dtype: float64

In [221]:
test[test["index"] = "-9BOs3rdWFMot0tlky983Q"]

,index,0,1,2,3,4,5,6,7,8,...,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
0,-9BOs3rdWFMot0tlky983Q,0.0,0.0,0.0,0.0,0.0,0.221574,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [180]:
cosine_similarities = cosine_similarity(np.array([test_df.loc["yup"]]), tfidf_matrix)

In [178]:
np.array([test_df.loc["yup"]])

array([[0., 0., 0., ..., 0., 0., 0.]])

In [67]:
len(user_profiles["--RlSfc-QmcHFGHyX6aVjA"][0])

5000

In [167]:
pd.DataFrame.from_dict(user_profiles, orient='index')

AttributeError: 'dict' object has no attribute 'ravel'

In [ ]:
test_user = user_profiles["--2vR0DIsmQ6WfcSzKWigw"]

pd.DataFrame(sorted(zip(tfidf_feature_names, 
                        user_profiles["--2vR0DIsmQ6WfcSzKWigw"].flatten().tolist()), key=lambda x: -x[1])[:20],
             columns=['token', 'relevance'])

In [ ]:
class ContentBasedRecommender:
    
    MODEL_NAME = 'Content-Based'
    
    def __init__(self, items_df=None):
        self.item_ids = item_ids
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def _get_similar_items_to_user_profile(self, user_id, topn=1000):
        #Computes the cosine similarity between the user profile and all item profiles
        cosine_similarities = cosine_similarity(user_profiles[user_id], tfidf_matrix)
        #Gets the top similar items
        similar_indices = cosine_similarities.argsort().flatten()[-topn:]
        #Sort the similar items by similarity
        similar_items = sorted([(item_ids[i], cosine_similarities[0,i]) for i in similar_indices], key=lambda x: -x[1])
        return similar_items
        
    def recommend(self, user_id, topn=10):
        
        reviewed_business_df = user_active_df[user_active_df["user_id"] == user_id]
        business_df_mod = business_df.rename(columns={"stars": "stars_business"})
        reviewed_business_df = pd.merge(reviewed_business_df, business_df_mod, how='left', left_on="business_id", right_on="business_id")
        reviewed_business_df = reviewed_business_df[["business_id", "name", "stars"]]
        already_reviewed = reviewed_business_df["business_id"]
        
        items_to_ignore = already_reviewed.values
        
        similar_items = self._get_similar_items_to_user_profile(user_id)
        #Ignores items the user has already interacted
        similar_items_filtered = list(filter(lambda x: x[0] not in items_to_ignore, similar_items))
        
        recommendations_df = pd.DataFrame(similar_items_filtered, columns=['business_id', 'rec_score']) \
                                    .head(topn)
        
        recommendations_df = pd.merge(recommendations_df, business_df, how='left', left_on="business_id", right_on="business_id")
        recommendations_df = recommendations_df[["business_id", "name", "categories", "attributes"]]

        return recommendations_df

In [ ]:
content_based_rec = ContentBasedRecommender(business_df)

In [ ]:
content_based_rec.recommend("--2vR0DIsmQ6WfcSzKWigw")

# Collaborative Filtering

In [6]:
review_df = pd.read_csv("/home/hongphuc95/notebookteam/fu/review.csv")

In [7]:
#review_df = pd.read_json(data_path + "cleaned/review_cleaned_2016_2019_Vegas.json", lines=True)

In [8]:
n_active = 40
user_review_df_count = review_df.groupby("user_id").size()
user_review_active_values = user_review_df_count[user_review_df_count >= n_active].reset_index()[
    "user_id"].values
user_active_df = review_df[review_df["user_id"].isin(user_review_active_values)]
user_active_df = user_active_df.groupby(["user_id", "business_id"], as_index=False).mean()

In [9]:
user_active_df.shape

(519838, 6)

In [10]:
user_active_df

,user_id,business_id,cool,useful,funny,stars
0,--2HUmLkcNHZp0xw6AMBPg,-ZBfr1BHvArFp1d6XH8jOQ,0.0,1.0,0.0,5.0
1,--2HUmLkcNHZp0xw6AMBPg,-vw3-EFSXYuNcRSr9SzwoA,0.0,1.0,0.0,5.0
2,--2HUmLkcNHZp0xw6AMBPg,3-6biVwm7VwPZ-k67jml-Q,0.0,4.0,0.0,2.0
3,--2HUmLkcNHZp0xw6AMBPg,4dCOilGYflzGzizOPKyvow,0.0,0.0,0.0,5.0
4,--2HUmLkcNHZp0xw6AMBPg,4j0WfdS_lHG9uSH7cd1PFg,0.0,0.0,0.0,4.0
...,...,...,...,...,...,...
519833,zzo--VpSQh8PpsGVeMC1dQ,r03BNov4Uyul5bJzlgWdJQ,0.0,3.0,0.0,4.0
519834,zzo--VpSQh8PpsGVeMC1dQ,tEfvf1cznLZUIeTcKKpAIw,1.0,0.0,0.0,1.0
519835,zzo--VpSQh8PpsGVeMC1dQ,vHz2RLtfUMVRPFmd7VBEHA,3.0,5.0,3.0,4.0
519836,zzo--VpSQh8PpsGVeMC1dQ,wxL9wgxLeuMMfI6SAXIzJw,0.0,5.0,0.0,5.0


In [11]:
from sklearn.preprocessing import LabelEncoder
user_encoder = LabelEncoder()
business_encoder = LabelEncoder()

user_ids = pd.DataFrame(columns=["user_id_matrix"])
business_ids = pd.DataFrame(columns=["business_id_matrix"])
user_active_df["user_id_matrix"] = user_encoder.fit_transform(user_active_df['user_id'])
user_active_df["business_id_matrix"] = business_encoder.fit_transform(user_active_df['business_id'])

user_ids["user_id_matrix"] = user_active_df["user_id_matrix"].unique()
business_ids["business_id_matrix"] = user_active_df["business_id_matrix"].unique()
user_ids["user_id"] = user_encoder.inverse_transform(user_ids["user_id_matrix"])
business_ids["business_id"] = business_encoder.inverse_transform(business_ids["business_id_matrix"])

In [12]:
highest_user_id = len(user_ids['user_id'].unique())
highest_business_id = len(business_ids['business_id'].unique())
shape_matrix = (highest_user_id, highest_business_id)
ratings_mat = sparse.lil_matrix(shape_matrix)

In [13]:
ratings_mat

<7024x90744 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in List of Lists format>

In [14]:
for i, row in user_active_df.iterrows():
    ratings_mat[row["user_id_matrix"], row["business_id_matrix"]] = row["stars"]

In [15]:
ratings_mat

<7024x90744 sparse matrix of type '<class 'numpy.float64'>'
	with 519838 stored elements in List of Lists format>

In [16]:
ratings_mat = ratings_mat.toarray()

In [17]:
ratings_mat.shape

(7024, 90744)

In [18]:
ratings_mat[ratings_mat == 0] = np.nan
user_ratings_mean = np.nanmean(ratings_mat, axis=1)

In [19]:
ratings_mat = np.where(np.isnan(ratings_mat), user_ratings_mean.reshape(-1,1), ratings_mat)

In [20]:
unique_elements, counts_elements = np.unique(ratings_mat[0], return_counts=True)

In [21]:
user_ratings_mean = np.mean(ratings_mat, axis=1)

In [22]:
ratings_mat_demeaned = ratings_mat - user_ratings_mean.reshape(-1, 1)

In [23]:
ratings_mat_demeaned

array([[ 1.77635684e-15,  1.77635684e-15,  1.77635684e-15, ...,
         1.77635684e-15,  1.77635684e-15,  1.77635684e-15],
       [ 8.88178420e-16,  8.88178420e-16,  8.88178420e-16, ...,
         8.88178420e-16,  8.88178420e-16,  8.88178420e-16],
       [ 4.44089210e-16,  4.44089210e-16,  4.44089210e-16, ...,
         4.44089210e-16,  4.44089210e-16,  4.44089210e-16],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-8.88178420e-16, -8.88178420e-16, -8.88178420e-16, ...,
        -8.88178420e-16, -8.88178420e-16, -8.88178420e-16],
       [-1.77635684e-15, -1.77635684e-15, -1.77635684e-15, ...,
        -1.77635684e-15, -1.77635684e-15, -1.77635684e-15]])

In [24]:
from scipy.sparse.linalg import svds
NUMBER_OF_FACTORS_MF = 20
U, sigma, Vt = svds(ratings_mat_demeaned, k = NUMBER_OF_FACTORS_MF)
sigma = np.diag(sigma)
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)

In [26]:
U.shape

(7024, 20)

In [27]:
U_df = pd.DataFrame(U)

In [28]:
user_similarity = cosine_similarity(U_df)

In [29]:
user_similarity_df = pd.DataFrame(user_similarity, index=U_df.index, columns=U_df.index)

In [30]:
user_similarity_df

,0,1,2,3,4,5,6,7,8,9,...,7014,7015,7016,7017,7018,7019,7020,7021,7022,7023
0,1.000000,0.005446,-0.322602,0.028582,-0.014740,0.043309,-0.342109,-0.094330,-0.143614,-0.020505,...,-0.025407,-0.094188,-0.018498,0.145564,-0.305288,-0.322669,-0.448930,0.769142,0.021764,0.031263
1,0.005446,1.000000,-0.009863,0.401679,0.476436,0.066175,0.021127,-0.065124,0.022259,0.258244,...,0.250039,-0.033582,-0.010401,0.021499,-0.003882,-0.037501,-0.170580,-0.030140,0.809500,0.762764
2,-0.322602,-0.009863,1.000000,-0.004653,0.007966,-0.021660,-0.264763,0.663377,0.162518,0.014729,...,0.005065,0.582545,-0.031525,0.050775,-0.051039,-0.288491,0.243191,-0.290326,-0.023460,-0.019509
3,0.028582,0.401679,-0.004653,1.000000,-0.443427,0.052066,0.011511,-0.038866,0.024887,-0.117648,...,0.235255,0.009177,-0.037755,0.118878,0.002480,-0.003296,-0.097640,-0.031210,0.521147,-0.021496
4,-0.014740,0.476436,0.007966,-0.443427,1.000000,-0.000616,0.005011,0.025816,-0.049543,0.567382,...,0.089465,0.065421,0.028979,-0.098110,0.004396,-0.009555,0.040390,0.006479,-0.025359,0.614701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7019,-0.322669,-0.037501,-0.288491,-0.003296,-0.009555,-0.028176,0.051726,-0.095662,0.183572,0.054717,...,0.023371,-0.060579,-0.108660,-0.131262,-0.068453,1.000000,0.180258,-0.089058,-0.050247,-0.073893
7020,-0.448930,-0.170580,0.243191,-0.097640,0.040390,-0.014022,0.000591,0.438640,-0.020015,0.227203,...,0.140337,0.236002,0.025726,0.181469,0.714939,0.180258,1.000000,-0.002254,-0.279714,-0.234621
7021,0.769142,-0.030140,-0.290326,-0.031210,0.006479,0.022979,-0.066826,-0.038536,0.017841,0.016907,...,-0.014909,0.012771,-0.051436,-0.066554,0.070088,-0.089058,-0.002254,1.000000,-0.034040,-0.008892
7022,0.021764,0.809500,-0.023460,0.521147,-0.025359,0.077650,0.015933,-0.110284,0.054683,-0.235668,...,0.130199,-0.064767,-0.001027,0.089999,-0.008333,-0.050247,-0.279714,-0.034040,1.000000,0.630912


In [31]:
a = user_similarity_df[35].sort_values(ascending=False)[:20]

In [32]:
type(a)

pandas.core.series.Series

In [33]:
a_df = pd.DataFrame({"score":a}).reset_index()

In [34]:
a_df

,index,score
0,35,1.000000
1,6436,0.998693
2,5156,0.997569
3,5809,0.997226
4,1607,0.996508
5,1422,0.995723
6,2689,0.994950
7,2856,0.993549
8,6341,0.992461
9,6818,0.991948


In [35]:
user_id_index = pd.merge(left=a_df, right=user_ids, how="inner", left_on="index", right_on="user_id_matrix")

In [36]:
user_id_index.drop(columns=["index", "user_id_matrix"], inplace=True)

In [37]:
user_id_index

,score,user_id
0,1.000000,-HlCe-Ya-4CE2huuEk2_1g
1,0.998693,ukp5bUGNcvl2OWU5CPG7fw
2,0.997569,iXgbdUe2D53mYYrGcY9lPA
3,0.997226,ohL1RSq-Fa76nIYIX4849w
4,0.996508,DkvaAV7F4Y_rw040k0XGvQ
5,0.995723,By9qnb2xvnlh4wY7qOtKVg
6,0.994950,Nq6e5N8bjgD9B46O4va_zA
7,0.993549,PIG12rx9hiH4xf9_KBMj4Q
8,0.992461,tpPg4_1g3QY00L1QJyJkVw
9,0.991948,yFgEFHCRJC4vfPM6usClhw


In [38]:
user_id_index_continue = pd.merge(left=user_id_index, right=user_active_df, how="inner", on="user_id")

In [39]:
test_user_a = user_id_index_continue[user_id_index_continue["user_id"]=="1GbtKqRpDAfv13fUYIBBmA"]

In [40]:
test_user_a = test_user_a[["score", "user_id", "business_id", "stars"]]

In [41]:
test_user_b = user_id_index_continue[user_id_index_continue["user_id"]=="4ywoghefRTsbi1SzkR-fcA"]

In [42]:
test_user_b = test_user_b[["score", "user_id", "business_id", "stars"]]

In [43]:
test_commun = pd.merge(left=test_user_a, right=test_user_b, how="inner", on="business_id")

In [81]:
test_commun[0:40]

,score_x,user_id_x,business_id,stars_x,score_y,user_id_y,stars_y
0,1.0,1GbtKqRpDAfv13fUYIBBmA,QDeB3ZtWYzVJJ0KK-HM-PA,5.0,0.793994,4ywoghefRTsbi1SzkR-fcA,5.0
1,1.0,1GbtKqRpDAfv13fUYIBBmA,ZgWjkMSzIV9fAuJJQXSUvA,4.0,0.793994,4ywoghefRTsbi1SzkR-fcA,4.0
2,1.0,1GbtKqRpDAfv13fUYIBBmA,q3oJ6bNRV3OoJrwc95GOwg,4.0,0.793994,4ywoghefRTsbi1SzkR-fcA,4.0


In [63]:
new_df = pd.merge(left=test_res, right=business_df, how="inner", on="business_id")

In [65]:
new_df[["user_id", "business_id", "stars_x", "stars_y", "name", "categories"]]

,user_id,business_id,stars_x,stars_y,name,categories
0,-9RU4LuI_TfYgv9rBijJoQ,-6c_bJblLXUwoWfmYLtH_w,1.0,2.5,Neo-Paws International,"Pet Services, Pet Sitting, Pet Groomers, Pet S..."
1,-9RU4LuI_TfYgv9rBijJoQ,1xrGLzRF2JCGSdBY4xh1xg,5.0,4.0,Pollo Loco Churrasqueria,"Restaurants, Portuguese"
2,-9RU4LuI_TfYgv9rBijJoQ,3S8nDh3gK2BmG-_HjDjh6w,2.0,3.5,Kothur Indian Cuisine,"Indian, Restaurants"
3,-9RU4LuI_TfYgv9rBijJoQ,3UU91ygbgRK0eS0I7IUrEw,5.0,4.5,Royal Myanmar Restaurant,"Burmese, Restaurants"
4,-9RU4LuI_TfYgv9rBijJoQ,4E6MLnW1Yh3TeKdHUtw5Qg,5.0,4.5,Domino Foods,"Food, Grocery, Specialty Food"
...,...,...,...,...,...,...
1389,vvlSoQlcekSfPm75T6ZRRA,pK06Y7dSR3JfqS1DgeSKtw,4.0,4.0,Byzantium,"Gay Bars, Bars, American (New), Canadian (New)..."
1390,vvlSoQlcekSfPm75T6ZRRA,utlVMNURJPDlsQfT-ulLdQ,5.0,5.0,Net Effect Internet Lounge,"Printing Services, Home Services, Professional..."
1391,vvlSoQlcekSfPm75T6ZRRA,vZyF3o30_m_-Pp5u0DkLoQ,5.0,3.5,Starving Artist,"Breakfast & Brunch, Food, Canadian (New), Rest..."
1392,vvlSoQlcekSfPm75T6ZRRA,wKmDYX_XuYCbIerCbYfWnQ,3.0,3.5,The Captain's Boil,"Asian Fusion, Seafood, Restaurants, Cajun/Creole"


In [18]:
all_user_predicted_ratings.shape

(7024, 90744)

In [19]:
all_user_predicted_ratings

matrix([[ 2.80572478e-05, -3.06430879e-04, -7.62462615e-04, ...,
         -8.92782792e-04, -1.19342250e-03,  2.50393074e-04],
        [-4.05617430e-04, -1.57598950e-03,  1.14617587e-03, ...,
         -6.23662393e-04, -1.35499455e-03, -4.97264856e-04],
        [-7.10023843e-05, -2.94977877e-05,  4.15602000e-06, ...,
          2.98501478e-03, -5.45721588e-05, -1.07575741e-03],
        ...,
        [ 1.28020963e-04, -7.67523658e-05, -8.58576604e-04, ...,
         -2.37807115e-03, -8.00979523e-04,  9.76637589e-04],
        [ 1.64032159e-03,  2.01158188e-02,  5.88522429e-04, ...,
          1.62551568e-03,  1.43704570e-04,  1.19472070e-03],
        [ 4.72845026e-04,  2.85724191e-02, -6.24569761e-04, ...,
          3.34162209e-04,  1.90163201e-04,  3.64881264e-04]])

In [18]:
from sklearn.decomposition import NMF
NUMBER_OF_FACTORS_MF = 20
nmf = NMF(n_components=NUMBER_OF_FACTORS_MF)
nmf.fit(ratings_mat)
#Features
W = nmf.transform(ratings_mat)
#Features Weight
H = nmf.components_
#Reconstructed matrix
ratings_mat_fitted = W.dot(H)

In [19]:
user_id = "AyjqBovADgbskmLrIBOMlQ"

In [21]:
user_id_num = user_ids[user_ids["user_id"] == user_id].user_id_matrix.values
if user_id_num.size > 0:
    user_id_num = user_id_num.astype(int)[0]
    pred_user = ratings_mat_fitted[user_id_num, :]
    sorted_user_predictions = pd.DataFrame(pred_user.T, columns=["score"])
    sorted_user_predictions = pd.merge(sorted_user_predictions, business_ids, left_index=True,
                                       right_on="business_id_matrix") \
        .drop(columns=["business_id_matrix"]) \
        .sort_values(ascending=False, by="score") \
        .reset_index(drop=True)

    items_to_ignore = user_active_df[user_active_df["user_id"] == user_id]
    items_to_ignore = items_to_ignore["business_id"].unique()

    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    nmf_rec_df = sorted_user_predictions[~sorted_user_predictions['business_id'].isin(items_to_ignore)] \
        .sort_values('score', ascending=False)

In [33]:
nmf_rec_df.reset_index(drop=True).head(20)

,nmf_score,business_id
0,2.126611,eaNenRk_liZBERFFLCXqqQ
1,2.026556,uBdYMY6a6A7FyxzTSwOiDg
2,2.017999,iFCz-xI7CV98fcaB4Chh3g
3,1.987034,L1-1P3acJc4gEFvWwjXcNQ
4,1.985794,i9D9xPBV0gR1Ja9kbY4NCw
5,1.821615,JPfi__QJAaRzmfh5aOyFEw
6,1.754412,YILyHegzhy1vlc_LNVfObw
7,1.672821,2iTsRqUsPGRH1li1WVRvKQ
8,1.654355,jaSowNITPRRCYpPb3_pjdA
9,1.590757,igHYkXZMLAc9UdV5VnR_AA


In [24]:
user_ratings_mean = np.mean(ratings_mat, axis = 1)
ratings_mat_demeaned = ratings_mat - user_ratings_mean.reshape(-1, 1)

In [25]:
from scipy.sparse.linalg import svds
NUMBER_OF_FACTORS_MF = 20
U, sigma, Vt = svds(ratings_mat_demeaned, k = NUMBER_OF_FACTORS_MF)
sigma = np.diag(sigma)
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)

In [26]:
user_id_num = user_ids[user_ids["user_id"] == user_id].user_id_matrix.values
if user_id_num.size > 0:
    user_id_num = user_id_num.astype(int)[0]
    pred_user = all_user_predicted_ratings[user_id_num, :]
    sorted_user_predictions = pd.DataFrame(pred_user.T, columns=["score"])
    sorted_user_predictions = pd.merge(sorted_user_predictions, business_ids, left_index=True,
                                       right_on="business_id_matrix") \
        .drop(columns=["business_id_matrix"]) \
        .sort_values(ascending=False, by="score") \
        .reset_index(drop=True)

    items_to_ignore = user_active_df[user_active_df["user_id"] == user_id]
    items_to_ignore = items_to_ignore["business_id"].unique()

    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    svd_rec_df = sorted_user_predictions[~sorted_user_predictions['business_id'].isin(items_to_ignore)] \
        .sort_values('score', ascending=False)

In [27]:
nmf_rec_df = nmf_rec_df.rename(columns={'score': 'nmf_score'})
svd_rec_df = svd_rec_df.rename(columns={'score': 'svd_score'})

In [28]:
recs_df = nmf_rec_df.merge(svd_rec_df,
                                   how = 'outer', 
                                   left_on = 'business_id', 
                                   right_on = 'business_id').fillna(0.0)

In [29]:
recs_df

,nmf_score,business_id,svd_score
0,2.126611,eaNenRk_liZBERFFLCXqqQ,2.643354
1,2.026556,uBdYMY6a6A7FyxzTSwOiDg,2.426658
2,2.017999,iFCz-xI7CV98fcaB4Chh3g,1.961197
3,1.987034,L1-1P3acJc4gEFvWwjXcNQ,1.796299
4,1.985794,i9D9xPBV0gR1Ja9kbY4NCw,2.284643
...,...,...,...
90543,0.000000,W8xG9Bzk3TPPkBR5jx1pgg,-0.019696
90544,0.000000,ITFKHyVqm3Q4TvRhBGCfRQ,-0.002117
90545,0.000000,IT8amJTTW64XAre4ilKcjg,-0.006783
90546,0.000000,W925Osx_5w9Wa6uy9MuWrQ,-0.000706


In [19]:
user_id_num = user_ids[user_ids["user_id"] == "-1x8gXJnrI-FeZPNvnfbRg"].user_id_matrix.values.astype(int)[0]

In [26]:
ratings_mat_fitted[user_id_num,:].max()

0.1353049643277316

In [25]:
error

2653.146573220556

In [12]:
#user_item_matrix = user_active_df.pivot(
#    index='user_id',
#    columns='business_id',
#    values='stars'
#).fillna(0)

In [13]:
#user_item_matrix

In [14]:
#R = user_item_matrix.values
#user_ratings_mean = np.mean(R, axis = 1)
#R_demeaned = R - user_ratings_mean.reshape(-1, 1)

In [15]:
#from scipy.sparse.linalg import svds
#U, sigma, Vt = svds(R_demeaned, k = 30)
#sigma = np.diag(sigma)
#all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)

In [16]:
ratings_mat

<4879x83789 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in List of Lists format>

In [17]:
for i, row in user_active_df.iterrows():
    ratings_mat[row["user_id_matrix"], row["business_id_matrix"]] = row["stars"]

In [18]:
ratings_mat

<4879x83789 sparse matrix of type '<class 'numpy.float64'>'
	with 429166 stored elements in List of Lists format>

In [19]:
user_ratings_mean = np.mean(ratings_mat, axis = 1)
ratings_mat_demeaned = ratings_mat - user_ratings_mean.reshape(-1, 1)

In [20]:
from scipy.sparse.linalg import svds

In [21]:
NUMBER_OF_FACTORS_MF = 20

In [22]:
U, sigma, Vt = svds(ratings_mat_demeaned, k = NUMBER_OF_FACTORS_MF)
sigma = np.diag(sigma)
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
#all_user_predicted_ratings = (U.dot(sigma)).dot(Vt)

In [23]:
all_user_predicted_ratings

matrix([[ 2.90650137e-04, -1.25689745e-04, -6.87756916e-04, ...,
         -8.16895265e-04, -9.23685649e-04, -3.18154465e-04],
        [-5.97851365e-04,  2.06256761e-04,  1.21544415e-03, ...,
         -6.15919726e-04, -1.32262949e-03, -5.88409428e-04],
        [ 1.46681038e-03, -7.05734557e-04,  2.72197328e-03, ...,
          1.50644383e-03,  2.92287588e-04,  1.20316439e-03],
        ...,
        [ 1.16753923e-03,  9.20353323e-04, -3.82908477e-04, ...,
          2.97482183e-03,  5.10077798e-04,  2.33000888e-03],
        [-7.26397112e-06, -1.14716435e-04, -1.27645432e-03, ...,
          3.15590042e-04, -5.30403301e-04, -1.84002838e-03],
        [ 1.81404889e-03,  1.94967644e-02,  7.04278306e-04, ...,
          1.84397189e-03,  3.48655794e-04,  1.53209399e-03]])

In [ ]:
((all_user_predicted_ratings - ratings_mat_demeaned)**2).mean(axis=None)

In [44]:
all_user_predicted_ratings.min()

-3.193333669098166

In [31]:
ratings_mat.toarray()[0].max()

5.0

In [34]:
def norm(row):
    return (row - row.min()) / (row.max() - row.min())

In [35]:
all_user_predicted_ratings_norm = np.apply_along_axis(norm, 1, all_user_predicted_ratings)

array([0.24956259, 0.3302804 , 0.3350676 , ..., 0.32781116, 0.33859511,
       0.32851144])

In [40]:
user1 = all_user_predicted_ratings_norm[1]

In [41]:
max(user1)

1.0

In [ ]:
# Normalization
all_user_predicted_ratings_norm = (all_user_predicted_ratings - all_user_predicted_ratings.min()) / (
        all_user_predicted_ratings.max() - all_user_predicted_ratings.min())

In [22]:
all_user_predicted_ratings_norm

array([[0.34280791, 0.34280502, 0.34297724, ..., 0.34317938, 0.34282498,
        0.34278847],
       [0.34286844, 0.34273874, 0.342951  , ..., 0.3428172 , 0.34277625,
        0.34282116],
       [0.34282429, 0.34282394, 0.3427979 , ..., 0.34305854, 0.34280984,
        0.34276875],
       ...,
       [0.34282327, 0.34282163, 0.34284245, ..., 0.34290842, 0.34281567,
        0.34278927],
       [0.34282562, 0.34282909, 0.34281982, ..., 0.34282092, 0.34281821,
        0.34282287],
       [0.34281318, 0.34459925, 0.34275293, ..., 0.3428307 , 0.34282673,
        0.34282274]])

In [27]:
def rmse(true, pred):
    # this will be used towards the end
    x = true - pred
    return sum([xi * xi for xi in x]) / len(x)

In [31]:
all_user_predicted_ratings

array([[-1.29993121e-02,  1.64113885e-05,  7.88346820e-04, ...,
        -3.81752044e-04,  1.35715577e-03, -2.68833401e-04],
       [ 1.06746998e-01,  3.42047137e-03,  2.70874326e-03, ...,
         6.89709035e-03,  4.11297742e-03, -1.17372356e-02],
       [ 2.25619643e-01,  1.02590692e-02,  1.96931358e-02, ...,
        -6.09213153e-03,  1.39677837e-03,  3.63162052e-03],
       ...,
       [ 1.78108387e-01,  2.38199713e-03, -2.03342789e-03, ...,
        -8.13566233e-04,  5.88566061e-04, -1.20061501e-02],
       [ 4.12030757e-01,  5.37632193e-03, -8.13677808e-03, ...,
        -2.52131418e-03, -1.40099212e-03,  7.19835137e-03],
       [ 8.28570232e-02,  2.93062614e-03,  1.21026100e-03, ...,
         3.26813109e-03, -6.47903511e-04,  4.90651978e-03]])

In [20]:
all_user_predicted_ratings - ratings_mat

matrix([[-2.39608523e-04, -2.85370111e-04,  2.44517349e-03, ...,
          5.65008525e-03,  3.10145024e-05, -5.47834778e-04],
        [ 7.20138322e-04, -1.33634859e-03,  2.02909653e-03, ...,
         -9.22301774e-05, -7.41589875e-04, -2.94634665e-05],
        [ 2.01710170e-05,  1.45673701e-05, -3.98275341e-04, ...,
          3.73412310e-03, -2.08993883e-04, -8.60511612e-04],
        ...,
        [ 4.01626374e-06, -2.21094515e-05,  3.08039977e-04, ...,
          1.35397694e-03, -1.16561149e-04, -5.35035718e-04],
        [ 4.11497062e-05,  9.62194798e-05, -5.07165664e-05, ...,
         -3.33152964e-05, -7.62539537e-05, -2.44726895e-06],
        [-1.56006301e-04,  2.81623620e-02, -1.11121166e-03, ...,
          1.21703129e-04,  5.87614833e-05, -4.51220271e-06]])

In [ ]:
import math
math.sqrt((np.power((all_user_predicted_ratings - ratings_mat), 2)).mean(axis=None))

In [23]:
all_user_predicted_ratings_norm.shape

(2289, 14904)

In [24]:
ratings_mat.shape

(2289, 14904)

In [30]:
a = pd.DataFrame(all_user_predicted_ratings_norm)

In [32]:
a

,0,1,2,3,4,5,6,7,8,9,...,99104,99105,99106,99107,99108,99109,99110,99111,99112,99113
0,0.342808,0.342805,0.342977,0.339234,0.342828,0.342825,0.343284,0.342789,0.342841,0.342824,...,0.342841,0.342831,0.342571,0.344943,0.342964,0.342813,0.342838,0.343179,0.342825,0.342788
1,0.342868,0.342739,0.342951,0.342910,0.343140,0.342823,0.342798,0.342812,0.342796,0.342822,...,0.342899,0.342818,0.342816,0.342784,0.342821,0.342819,0.343567,0.342817,0.342776,0.342821
2,0.342824,0.342824,0.342798,0.343180,0.342822,0.342823,0.342747,0.342824,0.342817,0.342823,...,0.342816,0.342817,0.342847,0.344157,0.342873,0.342821,0.342822,0.343059,0.342810,0.342769
3,0.342842,0.342725,0.342883,0.342867,0.342963,0.342823,0.342815,0.342818,0.342817,0.342823,...,0.342774,0.342821,0.342820,0.342809,0.342823,0.342822,0.343157,0.342819,0.342802,0.342822
4,0.342815,0.345048,0.342787,0.342830,0.342909,0.342823,0.342825,0.342826,0.342829,0.342823,...,0.342794,0.342820,0.342819,0.342823,0.342823,0.342822,0.342725,0.342822,0.342824,0.342824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10751,0.342809,0.342818,0.342953,0.339785,0.342825,0.342824,0.343228,0.342794,0.342853,0.342824,...,0.342843,0.342832,0.342612,0.343322,0.342877,0.342818,0.342832,0.343061,0.342835,0.342802
10752,0.342844,0.344023,0.342826,0.343036,0.342929,0.342823,0.342798,0.342823,0.342811,0.342823,...,0.343942,0.342835,0.342816,0.342791,0.342823,0.342817,0.342949,0.342813,0.342792,0.342822
10753,0.342823,0.342822,0.342842,0.342658,0.342825,0.342824,0.342819,0.342824,0.342818,0.342823,...,0.342821,0.342826,0.342792,0.345177,0.342947,0.342819,0.342822,0.342908,0.342816,0.342789
10754,0.342826,0.342829,0.342820,0.342843,0.342812,0.342823,0.342819,0.342823,0.342820,0.342823,...,0.343103,0.342827,0.342821,0.342817,0.342823,0.342822,0.342807,0.342821,0.342818,0.342823


In [33]:
user_ids

,user_id_matrix,user_id
0,0,--2HUmLkcNHZp0xw6AMBPg
1,1,--BumyUHiO_7YsHurb9Hkw
2,2,--Nnm_506G_p8MxAOQna5w
3,3,--Qh8yKWAvIP4V4K8ZPfHA
4,4,--YhjyV-ce1nFLYxP49C5A
...,...,...
10751,10751,zyebSPCZLUZHapi-dSHU5Q
10752,10752,zyg4-MFtfPWmwucVazSjfw
10753,10753,zyh_AzbO1JNnhywem3hUPg
10754,10754,zzKsgIF472IyDpJxmHDIpw


In [34]:
user_id_num = user_ids[user_ids["user_id"] == "-1x8gXJnrI-FeZPNvnfbRg"].user_id_matrix.values.astype(int)[0]

In [35]:
b = all_user_predicted_ratings_norm[user_id_num,:]

In [37]:
sorted_user_predictions = pd.DataFrame(b, columns=["score"])

In [38]:
sorted_user_predictions

,score
0,0.342824
1,0.342820
2,0.342810
3,0.343043
4,0.342822
...,...
99109,0.342824
99110,0.342822
99111,0.342827
99112,0.342809


In [40]:
sorted_user_predictions = pd.merge(sorted_user_predictions, business_ids, left_index=True,
                                               right_on="business_id_matrix") \

In [57]:
voted = sorted_user_predictions.sort_values(by="score", ascending=False).head(50)["business_id"].values

In [58]:
rated= user_active_df[user_active_df["user_id"] == "-1x8gXJnrI-FeZPNvnfbRg"]["business_id"].values

In [59]:
np.intersect1d(voted, rated)

array([], dtype=object)

In [55]:
sorted_user_predictions = pd.merge(c, business_ids, left_index=True, right_on="business_id_matrix") \
                        .drop(columns=["business_id_matrix"]) \
                        .sort_values(ascending=False, by="score") \
                        .reset_index(drop=True).head(20)

In [56]:
sorted_user_predictions

,score,business_id
0,0.595339,FaHADZARwnY4yvlvpnsfGA
1,0.582417,na4Th5DrNauOv-c43QQFvA
2,0.581801,Wxxvi3LZbHNIDwJ-ZimtnA
3,0.579028,JAmQCmczUclNUfZjkNdjQA
4,0.578397,VyjyHoBg3KC5BSFRlD0ZPQ
5,0.576692,3xmfT7l3xNH5LK1dLzfvGw
6,0.576240,5LNZ67Yw9RD6nf4_UhXOjw
7,0.575598,NCFwm2-TDb-oBQ2medmYDg
8,0.574682,qjnJFZtsY_nfRzoL3J_UWQ
9,0.573128,t-o_Sraneime4DDhWrQRBA


In [52]:
class CFRecommender:
    
    MODEL_NAME = 'Collaborative Filtering'
    
    def __init__(self, predictions_df, items_df=None):
        self.predictions_df = predictions_df
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend(self, user_id, items_to_ignore=[], topn=10):
        cleaned_out = all_user_predicted_ratings_norm[user_id,:]



In [ ]:
cf_recommender_model = CFRecommender(cf_preds_df, user_active_df)

In [ ]:
a = cf_recommender_model.recommend("--2vR0DIsmQ6WfcSzKWigw", ignore)

In [ ]:
a

# Hybird

In [ ]:
class HybridRecommender:
    
    MODEL_NAME = 'Hybrid'
    
    def __init__(self, cb_rec_model, cf_rec_model, items_df, cb_ensemble_weight=1.0, cf_ensemble_weight=1.0):
        self.cb_rec_model = cb_rec_model
        self.cf_rec_model = cf_rec_model
        self.cb_ensemble_weight = cb_ensemble_weight
        self.cf_ensemble_weight = cf_ensemble_weight
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend(self, user_id, items_to_ignore=[], topn=10):
        #Getting the top-1000 Content-based filtering recommendations
        cb_recs_df = self.cb_rec_model.recommend(user_id, items_to_ignore=items_to_ignore,
                                                           topn=1000).rename(columns={'rec_score': 'rec_score_cb'})
        
        #Getting the top-1000 Collaborative filtering recommendations
        cf_recs_df = self.cf_rec_model.recommend(user_id, items_to_ignore=items_to_ignore, 
                                                           topn=1000).rename(columns={'rec_score': 'rec_score_cf'})
        
        #Combining the results by contentId
        recs_df = cb_recs_df.merge(cf_recs_df,
                                   how = 'outer', 
                                   left_on = 'business_id', 
                                   right_on = 'business_id').fillna(0.0)
        
        #Computing a hybrid recommendation score based on CF and CB scores
        #recs_df['recStrengthHybrid'] = recs_df['recStrengthCB'] * recs_df['recStrengthCF'] 
        recs_df['rec_score_hybrid'] = (recs_df['rec_score_cb'] * self.cb_ensemble_weight) \
                                     + (recs_df['rec_score_cf'] * self.cf_ensemble_weight)
        
        #Sorting recommendations by hybrid score
        recommendations_df = recs_df.sort_values('rec_score_hybrid', ascending=False).head(topn)

        return recommendations_df

In [ ]:
hybrid_rec = HybridRecommender(content_based_rec, cf_recommender_model, business_df,
                                             cb_ensemble_weight=1.0, cf_ensemble_weight=100.0)

In [ ]:
hybrid_rec.recommend("--2vR0DIsmQ6WfcSzKWigw", topn=20)